In [ ]:
import tensorflow as tf 
tf.compat.v1.disable_eager_execution()
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tf.compat.v1.reset_default_graph()
    
#scene_t = get_element_gp_times([0,scene_duration], steps["t"])
batch_size=1; in_channels=2; 
in_height=None;#none;#could use scene_duration as input to "define neural net" then this equals length(scene_t); 
in_width=1; #width=frequency 

gp = tf.compat.v1.placeholder(dtype=tf.float64, shape=(batch_size, in_height, in_width, in_channels)) #tf.placeholder
mask = tf.compat.v1.placeholder(dtype=tf.float64, shape=(batch_size, in_height, 1, 1))

#Defining network
filter_height = 3; filter_width = 1; 
out_channels = 4; n_params = 8;
filters = [1,2,4]
weights = []; layer_inputs = [gp];
for i in range(len(filters)):
    W = tf.compat.v1.get_variable("W{}".format(i), 
            [filter_height, filter_width, 
            in_channels if i == 0 else out_channels, 
            n_params if i == len(filters)-1 else out_channels], 
            dtype=tf.float64, 
            initializer=tf.compat.v1.initializers.glorot_normal())
    b = tf.compat.v1.get_variable("b{}".format(i), 
            [n_params if i == len(filters)-1 else out_channels], 
            dtype=tf.float64, 
            initializer=tf.constant_initializer(value=0.0))
    conv_dilations = [filters[i], 1]
    L1 = tf.compat.v1.nn.convolution(layer_inputs[i], W, padding="SAME", 
         dilation_rate=conv_dilations, data_format="NHWC")
    #L1 = tf.nn.conv2d(gp, W, (1,), padding="SAME", data_format="NHWC")
    L1 = tf.nn.bias_add(L1, b, data_format="NHWC")
    C = tf.nn.relu(L1)
    layer_inputs.append(C)

#Convert maske convolutions to parameter estimates
tile_mask = tf.tile(mask, (1, 1, 1, n_params))
masked_conv = tf.multiply(tile_mask, layer_inputs[-1])
O = tf.reduce_mean(masked_conv,axis=[1,2]) #batch_size, n_params, 1
O = tf.nn.relu(O)
Wf = tf.compat.v1.get_variable("Wfinal",[1, n_params, n_params],dtype=tf.float64,
        initializer=tf.compat.v1.initializers.glorot_normal())
b = tf.compat.v1.get_variable("bfinal",[n_params],dtype=tf.float64, 
    initializer=tf.constant_initializer(value=0.0)) #constant_initializer used to have dtype
Wf = tf.tile(Wf, [batch_size, 1, 1])
M = tf.matmul(Wf, tf.expand_dims(O,2))
dist_parameters = tf.nn.bias_add(tf.squeeze(M,2), b)
batch_loss = tf.reduce_sum(tf.abs(dist_parameters),axis=1)
loss = tf.reduce_mean(batch_loss, axis=0)

In [ ]:
init=tf.compat.v1.global_variables_initializer()
sess = tf.compat.v1.Session()
sess.run(init)
result = sess.run(loss, feed_dict={gp:np.ones((1,6,1,2)), mask:np.random.randn(1,6,1,1)})

In [ ]:
print("Result: {}".format(result))
print("Shape of output: {}".format(np.shape(result)))

In [ ]:
variables = tf.compat.v1.trainable_variables()
gradients = tf.compat.v1.gradients(loss, variables)
optimizer = tf.compat.v1.train.AdamOptimizer(0.0001,beta1=0.9, beta2=0.999,epsilon=1e-8)
train_op = optimizer.apply_gradients(zip(gradients, variables))

In [ ]:
sess.run(tf.compat.v1.global_variables_initializer())
loss_result, _ = sess.run([loss, train_op], feed_dict={gp:np.ones((1,6,1,2)), mask:np.random.randn(1,6,1,1)})

In [ ]:
print(loss_result)

In [ ]:
x = [];
sess.run(tf.compat.v1.global_variables_initializer())
for j in range(10000):
    loss_result, d, _ = sess.run([loss, dist_parameters, train_op], feed_dict={gp:np.ones((1,6,1,2)), mask:np.random.randn(1,6,1,1)})
    x.append(loss_result)

In [ ]:
plt.plot(x)